In [ ]:
import json
import requests
import datetime
import uuid
import traceback
import base64
from websocket import create_connection
# base和token會在cmd(命令提示元)，啟動jupyter notebook時出現
notebook_path = '/test_01.ipynb'
base = 'http://127.0.0.1:10001'
headers = {'Authorization': 'Token zhejianzhang'}

In [ ]:
url = base + '/api/sessions'
params = '{"path":\"%s\","type":"notebook","name":"","kernel":{"id":null,"name":"python3"}}' % notebook_path
response = requests.post(url, headers=headers, data=params)
session = json.loads(response.text)
print(session)
kernel = session["kernel"]

In [ ]:
# 讀取notebook檔案，並獲取每個Cell裡的Code
url = base + '/api/contents' + notebook_path
response = requests.get(url,headers=headers)
file = json.loads(response.text)
code = [ c['source'] for c in file['content']['cells'] if len(c['source'])>0 ]
print(code)

In [ ]:
def send_execute_request(code):
    msg_id=  uuid.uuid1().hex
    print(msg_id)
    msg_type = 'execute_request'
    content = { 'code' : code, 'silent':False }
    hdr = { 'msg_id' :msg_id, 
        'username': 'test', 
        'session': uuid.uuid1().hex, 
        'data': datetime.datetime.now().isoformat(),
        'msg_type': msg_type,
        'version' : '5.0' }
    msg = { 'header': hdr, 'parent_header': hdr, 
        'metadata': {},
        'content': content }
    return msg

In [ ]:
# 開始啟動 WebSocket channels (request/reply)
ws = create_connection("ws://127.0.0.1:10001/api/kernels/"+kernel["id"]+"/channels?session_id"+session["id"], header=headers)
# for c in code:
#     ws.send(json.dumps(send_execute_request(c)))

In [ ]:
c = """
import matplotlib.pyplot as plt
import time

# 数据
x = [1, 2, 3, 4, 5]
y = [1, 4, 9, 16, 25]

# 创建折线图
plt.plot(x, y, marker='o')

# 添加标题和轴标签
plt.title("Simple Line Plot")
plt.xlabel("X Axis")
plt.ylabel("Y Axis")

# 显示图形
plt.grid(True)  # 显示网格线
plt.show()

x = [1, 2, 3, 4, 5]
y = [1, 4, 9, 16, 25]

# 创建折线图
plt.plot(x, y, marker='o')

# 添加标题和轴标签
plt.title("Simple Line Plot")
plt.xlabel("X Axis")
plt.ylabel("Y Axis")

# 显示图形
plt.grid(True)  # 显示网格线
plt.show()

for i in x:
    print(i)
    time.sleep(1)
"""
code = [c]
# 我們只拿Code執行完的訊息結果，其他訊息將被忽略
for i in range(0, len(code)):
    ws.send(json.dumps(send_execute_request(code[i])))
    try:
        msg_type = ''
        while True:
            rsp = json.loads(ws.recv())
            print(rsp)
            msg_type = rsp["msg_type"]
            # print(msg_type)
            # 顯示列印內容
            if msg_type == "stream":
                print(rsp["content"]["text"])
            elif msg_type == "execute_result":
                # 顯示圖片編碼
                if "image/png" in (rsp["content"]["data"].keys()):
                    
                    print(rsp["content"]["data"]["image/png"])
                # 顯示輸出結果
                else:
                    print(rsp["content"]["data"]["text/plain"])
                # 顯示計算表格
            elif msg_type == "display_data":
                image_data = rsp["content"]["data"]["image/png"]

                with open('output_image.png', 'wb') as f:
                    f.write(base64.b64decode(image_data))
                print('image saved')
                print(rsp["content"]["data"]["image/png"])
            # 顯示錯誤訊息
            elif msg_type == "error":
                print(rsp["content"]["traceback"])
            # 當狀態為idle，代表ws.recv()已經沒有任何訊息
            elif msg_type == "status" and rsp["content"]["execution_state"] == "idle":
                pass
    except:
            traceback.print_exc()
            ws.close()
            
ws.close()

In [ ]:
# ws = yield websocket_connect(ws_req)